<a href="https://colab.research.google.com/github/x1001000/Python_snippets/blob/master/sqlite3_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 新建一個關聯式資料庫的五個資料表: visits, patients, examiners, module1, module2

In [ ]:
import sqlite3
con = sqlite3.connect('example.db')
cur = con.cursor()
cur.execute('CREATE TABLE visits (id INTEGER PRIMARY KEY, datetime, patient_id REFERENCES patients(id), examiner_id REFERENCES examiners(id), module1_id REFERENCES module1(id), module2_id REFERENCES module2(id))')
cur.execute('CREATE TABLE patients (id PRIMARY KEY, name)')
cur.execute('CREATE TABLE examiners (id PRIMARY KEY, name)')
cur.execute('CREATE TABLE module1 (id INTEGER PRIMARY KEY, data1, data2, data3)')
cur.execute('CREATE TABLE module2 (id INTEGER PRIMARY KEY, data1, data2, data3, data4, data5)')

In [ ]:
#@title 重複執行此格，新增或更新醫檢員資料
員工編號 = "TW003" #@param {type:"string"}
員工姓名 = "\u67EF\u5B78\u5BB6" #@param {type:"string"}
try:
    cur.execute(f'INSERT INTO examiners (id, name) VALUES ("{員工編號}", "{員工姓名}")')
except:
    cur.execute(f'UPDATE examiners SET name = "{員工姓名}" WHERE id = "{員工編號}"')

### 檢視現有醫檢員: examiners資料表

In [ ]:
import pandas as pd
examiners = pd.read_sql('SELECT * FROM examiners', con)
examiners.style.hide_index()

id,name
TW001,英皇
TW002,郭董
TW003,柯學家


In [ ]:
#@title 重複執行此格，模擬醫檢員操作軟體
醫檢員姓名 = "\u90ED\u8463" #@param {type:"string"}
操作項目 = "\u50C5\u65B0\u589E\u6216\u66F4\u65B0\u75C5\u4EBA\u500B\u8CC7" #@param ["僅新增或更新病人個資", "使用 Module 1", "使用 Module 2"]
病人編號 = "QQ8888" #@param {type:"string"}
病人姓名 = "\u529B\u5B8F" #@param {type:"string"}
受檢日期 = "2022-01-14" #@param {type:"date"}
data1 =  22#@param {type:"number"}
data2 =  66#@param {type:"number"}
data3 =  101#@param {type:"number"}
data4 =  -1#@param {type:"number"}
data5 =  -2#@param {type:"number"}

if (醫檢員姓名 == examiners['name']).any():

    try:
        cur.execute(f'INSERT INTO patients (id, name) VALUES ("{病人編號}", "{病人姓名}")')
    except:
        cur.execute(f'UPDATE patients SET name = "{病人姓名}" WHERE id = "{病人編號}"')

    if 操作項目 == '使用 Module 1':
        cur.execute(f'INSERT INTO module1 (data1, data2, data3) VALUES ({data1}, {data2}, {data3})')
        cur.execute(f'INSERT INTO visits (datetime, patient_id, examiner_id, module1_id, module2_id) VALUES ("{受檢日期}", "{病人編號}", (SELECT id FROM examiners WHERE name = "{醫檢員姓名}"), (SELECT MAX(id) FROM module1), 0)')

    if 操作項目 == '使用 Module 2':
        cur.execute(f'INSERT INTO module2 (data1, data2, data3, data4, data5) VALUES ({data1}, {data2}, {data3}, {data4}, {data5})')
        cur.execute(f'INSERT INTO visits (datetime, patient_id, examiner_id, module1_id, module2_id) VALUES ("{受檢日期}", "{病人編號}", (SELECT id FROM examiners WHERE name = "{醫檢員姓名}"), 0, (SELECT MAX(id) FROM module2))')

else:
    print('沒有這位醫檢員！')

### 檢視visits資料表和patients資料表

In [ ]:
visits = pd.read_sql('SELECT * FROM visits', con)
visits.style.hide_index()

id,datetime,patient_id,examiner_id,module1_id,module2_id
1,2022-01-13,QQ5566,TW003,1,0
2,2022-01-13,QQ5566,TW003,0,1
3,2022-01-14,QQ8888,TW002,2,0


In [ ]:
patients = pd.read_sql('SELECT * FROM patients', con)
patients.style.hide_index()

id,name
QQ5566,Phil
QQ8888,力宏


### 檢視module1資料表和module2資料表

In [ ]:
module1 = pd.read_sql('SELECT * FROM module1', con)
module1.style.hide_index()

id,data1,data2,data3
1,100,1000,3.140000
2,22,66,101.000000


In [ ]:
module2 = pd.read_sql('SELECT * FROM module2', con)
module2.style.hide_index()

id,data1,data2,data3,data4,data5
1,9487,0.010000,100,-1,-2
